# Notebook for building a model for the Sendfrid sign up dataset
***
**Jake Mitchell Scott Schubert**

Initially using XGBoost and TensorFlow

In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split

Hopeful Workflow:
- build model
- have function that takes a pandas dataframe object or csv and returns a dataframe or csv of predictions.

In [42]:
train_path = 'data/signup_train_data.csv'
test_path = 'data/signup_test_data.csv'

# Load the data into a DataFrame 
df = pd.read_csv(train_path, low_memory=False, nrows=1000)
# Split into X and Y
dfLabels = df.pop("label")
#print(df.head(100)["banned_email"])
#get train and valid sets
X_train, X_valid, y_train, y_valid = train_test_split(df, dfLabels, train_size=0.5, random_state=1000)

C:\Users\jakem\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [111]:
df = pd.read_csv(train_path, low_memory=False)
#print("banned_ip label corr:", df['label'].corr(df['banned_ip']))
#print("banned_email label corr:", df['label'].corr(df['banned_email']))
df.corr()["label"]
print("banned but fine:", df.loc[df["label"] == 0].loc[df["banned_ip"]>0][["banned_ip","label"]].count)
print("banned and bad:", df.loc[df["label"] == 1].loc[df["banned_ip"]>0][["banned_ip","label"]].count)
print("not banned but bad:", df.loc[df["label"] == 1].loc[df["banned_ip"]==0][["banned_ip","label"]].count)
print("not banned but fine:", df.loc[df["label"] == 0].loc[df["banned_ip"]==0][["banned_ip","label"]].count)

banned but fine: <bound method DataFrame.count of         banned_ip  label
90              3      0
104             1      0
138             1      0
189             2      0
405             1      0
456             1      0
584             1      0
589             2      0
643             1      0
695             6      0
1018            1      0
1071            1      0
1082            1      0
1092            2      0
1113            4      0
1148            6      0
1151            6      0
1185            6      0
1191            3      0
1222           35      0
1319            5      0
1338            7      0
1360            2      0
1448            2      0
1451            1      0
1474            3      0
1483            1      0
1489            1      0
1628            8      0
1698            1      0
...           ...    ...
332151          1      0
332155          1      0
332233          1      0
332300          5      0
332391          1      0
332521          1      0


In [106]:
def get_csv_prediction(filepath, nrows=None, getAcc=False, outputFile="data/output.csv"):
    '''Takes a csv filepath, loads the data, and stores the predicted results'''
    df = pd.read_csv(filepath, low_memory=False, nrows=nrows)
    results = get_df_prediction(df, getAcc=getAcc)
    df[["id","predict_label"]].to_csv(outputFile, header=["id", "label"], index = False)
    
    '''outF = open("output.csv", "w", newline='')
    writer = csv.writer(outF, delimiter=',')
    writer.writerow(["id","input"])
    print(df.columns)
    writer.writerow([df[row[0], "id"], df[row[0], "predict_label"]])
    outF.close()'''

    
def get_df_prediction(df, getAcc=False):
    '''Takes a dataframe of inputs, loops through them to get the predictions, and returns a predicted result dataframe'''
    df['predict_label'] = 0
    numCorrect = 0
    for row in df.iterrows():
        #row takes the form of (index, data)
        label = get_label(row[1])
        df.at[row[0], "predict_label"] = label
        if getAcc:
            if df.at[row[0], "label"] == label:
                numCorrect += 1
    if getAcc:
        print("numCorrect", numCorrect)
        print("numRows", df.shape[0])
        print("accuracy", numCorrect/df.shape[0])
        
        
def get_label(X):
    '''Takes a row of X inputs and returns the predicted label'''
    if X["banned_ip"]: #this alone is ~92%
        return 1
    elif X["banned_email"] > 5:
        return 1
    return 0

In [107]:
get_csv_prediction(train_path, getAcc=True)

numCorrect 307410
numRows 333926
accuracy 0.9205931853165072


In [109]:
#produce Output for Kaggle
get_csv_prediction(test_path, outputFile="data/outputBIpBemGt5.csv")